In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:
train_beneficiary_df = pd.read_csv('/content/drive/MyDrive/Health care/Train_Beneficiarydata-1542865627584.csv')
train_beneficiary_df.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,...,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,...,1,1,1,2,1,1,36000,3204,60,70
1,BENE11002,1936-09-01,NaN,2,1,0,39,280,12,12,...,2,2,2,2,2,2,0,0,30,50
2,BENE11003,1936-08-01,NaN,1,1,0,52,590,12,12,...,2,2,1,2,2,2,0,0,90,40
3,BENE11004,1922-07-01,NaN,1,1,0,39,270,12,12,...,2,1,1,1,1,2,0,0,1810,760
4,BENE11005,1935-09-01,NaN,1,1,0,24,680,12,12,...,2,1,2,2,2,2,0,0,1790,1200


In [ ]:
print('There are ' + str(train_beneficiary_df.shape[0]) + ' rows and ' + str(train_beneficiary_df.shape[1]) + ' columns in the beneficiary details dataset.')

There are 138556 rows and 25 columns in the beneficiary details dataset.


In [ ]:
train_beneficiary_df = train_beneficiary_df.loc[:, (train_beneficiary_df != 0).any(axis=0)]

In [ ]:
train_beneficiary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138556 entries, 0 to 138555
Data columns (total 25 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   BeneID                           138556 non-null  object
 1   DOB                              138556 non-null  object
 2   DOD                              1421 non-null    object
 3   Gender                           138556 non-null  int64 
 4   Race                             138556 non-null  int64 
 5   RenalDiseaseIndicator            138556 non-null  object
 6   State                            138556 non-null  int64 
 7   County                           138556 non-null  int64 
 8   NoOfMonths_PartACov              138556 non-null  int64 
 9   NoOfMonths_PartBCov              138556 non-null  int64 
 10  ChronicCond_Alzheimer            138556 non-null  int64 
 11  ChronicCond_Heartfailure         138556 non-null  int64 
 12  ChronicCond_Kidn

In [ ]:
train_beneficiary_df['DOD'] = train_beneficiary_df.DOD.fillna(train_beneficiary_df.DOD.value_counts().index[0])

train_beneficiary_df['DOD'] = pd.to_datetime(train_beneficiary_df['DOD'], format = '%Y-%m-%d')
train_beneficiary_df['DOB'] = pd.to_datetime(train_beneficiary_df['DOB'], format = '%Y-%m-%d')
train_beneficiary_df['Age'] = ((train_beneficiary_df['DOD'] - train_beneficiary_df['DOB']).dt.days/365).astype('int64')

train_beneficiary_df = train_beneficiary_df.drop(columns = ['DOB', 'DOD'])

train_beneficiary_df.head()

,BeneID,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age
0,BENE11001,1,1,0,39,230,12,12,1,2,...,1,1,2,1,1,36000,3204,60,70,66
1,BENE11002,2,1,0,39,280,12,12,2,2,...,2,2,2,2,2,0,0,30,50,73
2,BENE11003,1,1,0,52,590,12,12,1,2,...,2,1,2,2,2,0,0,90,40,73
3,BENE11004,1,1,0,39,270,12,12,1,1,...,1,1,1,1,2,0,0,1810,760,87
4,BENE11005,1,1,0,24,680,12,12,2,2,...,1,2,2,2,2,0,0,1790,1200,74


In [ ]:
train_beneficiary_df.RenalDiseaseIndicator.unique()

array(['0', 'Y'], dtype=object)

In [ ]:
label_encoder = preprocessing.LabelEncoder()
train_beneficiary_df['RenalDiseaseIndicator'] = label_encoder.fit_transform(train_beneficiary_df['RenalDiseaseIndicator'])
train_beneficiary_df.RenalDiseaseIndicator.unique()

array([0, 1])

In [ ]:
train_beneficiary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138556 entries, 0 to 138555
Data columns (total 24 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   BeneID                           138556 non-null  object
 1   Gender                           138556 non-null  int64 
 2   Race                             138556 non-null  int64 
 3   RenalDiseaseIndicator            138556 non-null  int64 
 4   State                            138556 non-null  int64 
 5   County                           138556 non-null  int64 
 6   NoOfMonths_PartACov              138556 non-null  int64 
 7   NoOfMonths_PartBCov              138556 non-null  int64 
 8   ChronicCond_Alzheimer            138556 non-null  int64 
 9   ChronicCond_Heartfailure         138556 non-null  int64 
 10  ChronicCond_KidneyDisease        138556 non-null  int64 
 11  ChronicCond_Cancer               138556 non-null  int64 
 12  ChronicCond_Obst

In [ ]:
train_inPatient_df = pd.read_csv("/content/drive/MyDrive/Health care/Train_Inpatientdata-1542865627584.csv");
train_inPatient_df.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,5119,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
print('There are ' + str(train_inPatient_df.shape[0]) + ' rows and ' + str(train_inPatient_df.shape[1]) + ' columns in the in-patient dataset.')

There are 40474 rows and 30 columns in the in-patient dataset.


In [ ]:
train_inPatient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40474 entries, 0 to 40473
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   BeneID                  40474 non-null  object 
 1   ClaimID                 40474 non-null  object 
 2   ClaimStartDt            40474 non-null  object 
 3   ClaimEndDt              40474 non-null  object 
 4   Provider                40474 non-null  object 
 5   InscClaimAmtReimbursed  40474 non-null  int64  
 6   AttendingPhysician      40362 non-null  object 
 7   OperatingPhysician      23830 non-null  object 
 8   OtherPhysician          4690 non-null   object 
 9   AdmissionDt             40474 non-null  object 
 10  ClmAdmitDiagnosisCode   40474 non-null  object 
 11  DeductibleAmtPaid       39575 non-null  float64
 12  DischargeDt             40474 non-null  object 
 13  DiagnosisGroupCode      40474 non-null  object 
 14  ClmDiagnosisCode_1      40474 non-null

In [ ]:
train_inPatient_df['OperatingPhysician'].fillna('None', inplace = True)
train_inPatient_df['OtherPhysician'].fillna('None', inplace = True)
train_inPatient_df['ClaimStartDt'] = pd.to_datetime(train_inPatient_df['ClaimStartDt'], format = '%Y-%m-%d')
train_inPatient_df['ClaimEndDt'] = pd.to_datetime(train_inPatient_df['ClaimEndDt'], format = '%Y-%m-%d')
train_inPatient_df['ClaimDurationInDays'] = ((train_inPatient_df['ClaimEndDt'] - train_inPatient_df['ClaimStartDt']).dt.days).astype('int64')

train_inPatient_df['AdmissionDt'] = pd.to_datetime(train_inPatient_df['AdmissionDt'], format = '%Y-%m-%d')
train_inPatient_df['DischargeDt'] = pd.to_datetime(train_inPatient_df['DischargeDt'], format = '%Y-%m-%d')
train_inPatient_df['AdmissionDurationInDays'] = ((train_inPatient_df['DischargeDt'] - train_inPatient_df['AdmissionDt']).dt.days).astype('int64')

train_inPatient_df = train_inPatient_df.drop(columns = ['ClaimStartDt', 'ClaimEndDt', 'AdmissionDt', 'DischargeDt'])

train_inPatient_df.head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,...,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,ClaimDurationInDays,AdmissionDurationInDays
0,BENE11001,CLM46614,PRV55912,26000,PHY390922,None,None,7866,1068.0,201,...,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,6
1,BENE11001,CLM66048,PRV55907,5000,PHY318495,PHY318495,None,6186,1068.0,750,...,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN,2,2
2,BENE11001,CLM68358,PRV56046,5000,PHY372395,None,PHY324689,29590,1068.0,883,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3
3,BENE11011,CLM38412,PRV52405,5000,PHY369659,PHY392961,PHY349768,431,1068.0,067,...,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN,8,8
4,BENE11014,CLM63689,PRV56614,10000,PHY379376,PHY398258,None,78321,1068.0,975,...,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN,17,17


In [ ]:
train_inPatient_df.drop(columns = ['ClmProcedureCode_5', 'ClmProcedureCode_6'], axis=1, inplace= True)

for ClmProcedureCodeCount in range(4):
    train_inPatient_df['ClmProcedureCode_{}'.format(ClmProcedureCodeCount + 1)].fillna(0, inplace = True)
    
for ClmDiagnosisCodeCount in range(10):
    train_inPatient_df['ClmDiagnosisCode_{}'.format(ClmDiagnosisCodeCount + 1)].fillna('0', inplace = True)
print('Unique values in the DeductibleAmtPaid Column:', train_inPatient_df.DeductibleAmtPaid.unique())

Unique values in the DeductibleAmtPaid Column: [1068.   nan]


In [ ]:
train_inPatient_df.DeductibleAmtPaid.fillna(0, inplace = True)
train_inPatient_df.apply(lambda x: x.fillna(x.value_counts().index[0], inplace = True))
train_inPatient_df.head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,...,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClaimDurationInDays,AdmissionDurationInDays
0,BENE11001,CLM46614,PRV55912,26000,PHY390922,None,None,7866,1068.0,201,...,2724,19889,5849,0,0.0,0.0,0.0,0.0,6,6
1,BENE11001,CLM66048,PRV55907,5000,PHY318495,PHY318495,None,6186,1068.0,750,...,0,0,0,0,7092.0,0.0,0.0,0.0,2,2
2,BENE11001,CLM68358,PRV56046,5000,PHY372395,None,PHY324689,29590,1068.0,883,...,0,0,0,0,0.0,0.0,0.0,0.0,3,3
3,BENE11011,CLM38412,PRV52405,5000,PHY369659,PHY392961,PHY349768,431,1068.0,067,...,25062,40390,4019,0,331.0,0.0,0.0,0.0,8,8
4,BENE11014,CLM63689,PRV56614,10000,PHY379376,PHY398258,None,78321,1068.0,975,...,5119,29620,20300,0,3893.0,0.0,0.0,0.0,17,17


In [ ]:
train_inPatient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40474 entries, 0 to 40473
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BeneID                   40474 non-null  object 
 1   ClaimID                  40474 non-null  object 
 2   Provider                 40474 non-null  object 
 3   InscClaimAmtReimbursed   40474 non-null  int64  
 4   AttendingPhysician       40474 non-null  object 
 5   OperatingPhysician       40474 non-null  object 
 6   OtherPhysician           40474 non-null  object 
 7   ClmAdmitDiagnosisCode    40474 non-null  object 
 8   DeductibleAmtPaid        40474 non-null  float64
 9   DiagnosisGroupCode       40474 non-null  object 
 10  ClmDiagnosisCode_1       40474 non-null  object 
 11  ClmDiagnosisCode_2       40474 non-null  object 
 12  ClmDiagnosisCode_3       40474 non-null  object 
 13  ClmDiagnosisCode_4       40474 non-null  object 
 14  ClmDiagnosisCode_5    

In [ ]:
train_outPatient_df = pd.read_csv('/content/drive/MyDrive/Health care/Train_Outpatientdata-1542865627584.csv')
train_outPatient_df.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,56409
1,BENE11003,CLM189947,2009-02-12,2009-02-12,PRV57610,80,PHY362868,NaN,NaN,6115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,79380
2,BENE11003,CLM438021,2009-06-27,2009-06-27,PRV57595,10,PHY328821,NaN,NaN,2723,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,71947


In [ ]:
print('There are ' + str(train_outPatient_df.shape[0]) + ' rows and ' + str(train_outPatient_df.shape[1]) + ' columns in the out-patient dataset.')

There are 517737 rows and 27 columns in the out-patient dataset.


In [ ]:
train_outPatient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517737 entries, 0 to 517736
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   BeneID                  517737 non-null  object 
 1   ClaimID                 517737 non-null  object 
 2   ClaimStartDt            517737 non-null  object 
 3   ClaimEndDt              517737 non-null  object 
 4   Provider                517737 non-null  object 
 5   InscClaimAmtReimbursed  517737 non-null  int64  
 6   AttendingPhysician      516341 non-null  object 
 7   OperatingPhysician      90617 non-null   object 
 8   OtherPhysician          195046 non-null  object 
 9   ClmDiagnosisCode_1      507284 non-null  object 
 10  ClmDiagnosisCode_2      322357 non-null  object 
 11  ClmDiagnosisCode_3      203257 non-null  object 
 12  ClmDiagnosisCode_4      125596 non-null  object 
 13  ClmDiagnosisCode_5      74344 non-null   object 
 14  ClmDiagnosisCode_6  

In [ ]:
train_outPatient_df['OperatingPhysician'].fillna('None', inplace = True)
train_outPatient_df['OtherPhysician'].fillna('None', inplace = True)

In [ ]:
train_outPatient_df['ClaimStartDt'] = pd.to_datetime(train_outPatient_df['ClaimStartDt'], format = '%Y-%m-%d')
train_outPatient_df['ClaimEndDt'] = pd.to_datetime(train_outPatient_df['ClaimEndDt'], format = '%Y-%m-%d')
train_outPatient_df['ClaimDurationInDays'] = ((train_outPatient_df['ClaimEndDt'] - train_outPatient_df['ClaimStartDt']).dt.days).astype('int64')

train_outPatient_df = train_outPatient_df.drop(columns = ['ClaimStartDt', 'ClaimEndDt'])

train_outPatient_df.head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,...,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode,ClaimDurationInDays
0,BENE11002,CLM624349,PRV56011,30,PHY326117,None,None,78943,V5866,V1272,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,56409,0
1,BENE11003,CLM189947,PRV57610,80,PHY362868,None,None,6115,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,79380,0
2,BENE11003,CLM438021,PRV57595,10,PHY328821,None,None,2723,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0
3,BENE11004,CLM121801,PRV56011,40,PHY334319,None,None,71988,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0
4,BENE11004,CLM150998,PRV56011,200,PHY403831,None,None,82382,30000,72887,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,71947,0


In [ ]:
train_outPatient_df.drop(columns = ['ClmProcedureCode_3' , 'ClmProcedureCode_4', 'ClmProcedureCode_5', 'ClmProcedureCode_6'], axis=1, inplace= True)

for ClmProcedureCodeCount in range(2):
    train_outPatient_df['ClmProcedureCode_{}'.format(ClmProcedureCodeCount + 1)].fillna(0, inplace = True)
    
for ClmDiagnosisCodeCount in range(10):
    train_outPatient_df['ClmDiagnosisCode_{}'.format(ClmDiagnosisCodeCount + 1)].fillna('0', inplace = True)

In [ ]:
train_outPatient_df.apply(lambda x: x.fillna(x.value_counts().index[0], inplace = True))
train_outPatient_df.head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,...,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,DeductibleAmtPaid,ClmAdmitDiagnosisCode,ClaimDurationInDays
0,BENE11002,CLM624349,PRV56011,30,PHY326117,None,None,78943,V5866,V1272,...,0,0,0,0,0,0.0,0.0,0,56409,0
1,BENE11003,CLM189947,PRV57610,80,PHY362868,None,None,6115,0,0,...,0,0,0,0,0,0.0,0.0,0,79380,0
2,BENE11003,CLM438021,PRV57595,10,PHY328821,None,None,2723,0,0,...,0,0,0,0,0,0.0,0.0,0,V7612,0
3,BENE11004,CLM121801,PRV56011,40,PHY334319,None,None,71988,0,0,...,0,0,0,0,0,0.0,0.0,0,V7612,0
4,BENE11004,CLM150998,PRV56011,200,PHY403831,None,None,82382,30000,72887,...,V4577,0,0,0,0,0.0,0.0,0,71947,0


In [ ]:
train_outPatient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517737 entries, 0 to 517736
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   BeneID                  517737 non-null  object 
 1   ClaimID                 517737 non-null  object 
 2   Provider                517737 non-null  object 
 3   InscClaimAmtReimbursed  517737 non-null  int64  
 4   AttendingPhysician      517737 non-null  object 
 5   OperatingPhysician      517737 non-null  object 
 6   OtherPhysician          517737 non-null  object 
 7   ClmDiagnosisCode_1      517737 non-null  object 
 8   ClmDiagnosisCode_2      517737 non-null  object 
 9   ClmDiagnosisCode_3      517737 non-null  object 
 10  ClmDiagnosisCode_4      517737 non-null  object 
 11  ClmDiagnosisCode_5      517737 non-null  object 
 12  ClmDiagnosisCode_6      517737 non-null  object 
 13  ClmDiagnosisCode_7      517737 non-null  object 
 14  ClmDiagnosisCode_8  

In [ ]:
train_inPatientMerged = pd.merge(train_inPatient_df, train_beneficiary_df, on = 'BeneID', how = 'inner')
train_inPatientMerged.head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age
0,BENE11001,CLM46614,PRV55912,26000,PHY390922,None,None,7866,1068.0,201,...,1,1,2,1,1,36000,3204,60,70,66
1,BENE11001,CLM66048,PRV55907,5000,PHY318495,PHY318495,None,6186,1068.0,750,...,1,1,2,1,1,36000,3204,60,70,66
2,BENE11001,CLM68358,PRV56046,5000,PHY372395,None,PHY324689,29590,1068.0,883,...,1,1,2,1,1,36000,3204,60,70,66
3,BENE11011,CLM38412,PRV52405,5000,PHY369659,PHY392961,PHY349768,431,1068.0,067,...,1,2,2,1,1,5000,1068,250,320,95
4,BENE11014,CLM63689,PRV56614,10000,PHY379376,PHY398258,None,78321,1068.0,975,...,2,1,2,2,2,21260,2136,120,100,71


In [ ]:
train_inPatientMerged.shape

(40474, 49)

In [ ]:
train_inPatientMerged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40474 entries, 0 to 40473
Data columns (total 49 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   BeneID                           40474 non-null  object 
 1   ClaimID                          40474 non-null  object 
 2   Provider                         40474 non-null  object 
 3   InscClaimAmtReimbursed           40474 non-null  int64  
 4   AttendingPhysician               40474 non-null  object 
 5   OperatingPhysician               40474 non-null  object 
 6   OtherPhysician                   40474 non-null  object 
 7   ClmAdmitDiagnosisCode            40474 non-null  object 
 8   DeductibleAmtPaid                40474 non-null  float64
 9   DiagnosisGroupCode               40474 non-null  object 
 10  ClmDiagnosisCode_1               40474 non-null  object 
 11  ClmDiagnosisCode_2               40474 non-null  object 
 12  ClmDiagnosisCode_3

In [ ]:
train_outPatientMerged = pd.merge(train_outPatient_df, train_beneficiary_df, on = 'BeneID', how = 'inner')
train_outPatientMerged.head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age
0,BENE11002,CLM624349,PRV56011,30,PHY326117,None,None,78943,V5866,V1272,...,2,2,2,2,2,0,0,30,50,73
1,BENE11003,CLM189947,PRV57610,80,PHY362868,None,None,6115,0,0,...,2,1,2,2,2,0,0,90,40,73
2,BENE11003,CLM438021,PRV57595,10,PHY328821,None,None,2723,0,0,...,2,1,2,2,2,0,0,90,40,73
3,BENE11004,CLM121801,PRV56011,40,PHY334319,None,None,71988,0,0,...,1,1,1,1,2,0,0,1810,760,87
4,BENE11004,CLM150998,PRV56011,200,PHY403831,None,None,82382,30000,72887,...,1,1,1,1,2,0,0,1810,760,87


In [ ]:
train_outPatientMerged.shape

(517737, 45)

In [ ]:
train_outPatientMerged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 517737 entries, 0 to 517736
Data columns (total 45 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   BeneID                           517737 non-null  object 
 1   ClaimID                          517737 non-null  object 
 2   Provider                         517737 non-null  object 
 3   InscClaimAmtReimbursed           517737 non-null  int64  
 4   AttendingPhysician               517737 non-null  object 
 5   OperatingPhysician               517737 non-null  object 
 6   OtherPhysician                   517737 non-null  object 
 7   ClmDiagnosisCode_1               517737 non-null  object 
 8   ClmDiagnosisCode_2               517737 non-null  object 
 9   ClmDiagnosisCode_3               517737 non-null  object 
 10  ClmDiagnosisCode_4               517737 non-null  object 
 11  ClmDiagnosisCode_5               517737 non-null  object 
 12  Cl

In [ ]:
train_providerFraud_df = pd.read_csv('/content/drive/MyDrive/Health care/Train-1542865627584.csv')
train_providerFraud_df.head()

,Provider,PotentialFraud
0,PRV51001,No
1,PRV51003,Yes
2,PRV51004,No
3,PRV51005,Yes
4,PRV51007,No


In [ ]:
train_providerFraud_df.shape

(5410, 2)

In [ ]:
train_providerFraud_df['PotentialFraud'] = label_encoder.fit_transform(train_providerFraud_df['PotentialFraud'])
train_providerFraud_df.PotentialFraud.unique()

array([0, 1])

In [ ]:
train_inPatientMergedWithProviderFraud_df = pd.merge(train_inPatientMerged, train_providerFraud_df, on = 'Provider', how = 'inner')
train_inPatientMergedWithProviderFraud_df.head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,...,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age,PotentialFraud
0,BENE11001,CLM46614,PRV55912,26000,PHY390922,None,None,7866,1068.0,201,...,1,2,1,1,36000,3204,60,70,66,1
1,BENE17521,CLM34721,PRV55912,19000,PHY349293,PHY370861,PHY363291,45340,1068.0,987,...,1,2,2,2,19000,1068,100,20,96,1
2,BENE21718,CLM72336,PRV55912,17000,PHY334706,PHY334706,None,V5789,1068.0,941,...,1,1,2,2,17000,1068,1050,540,87,1
3,BENE22934,CLM73394,PRV55912,13000,PHY390614,PHY323689,PHY363291,71946,1068.0,506,...,1,1,1,2,27000,2136,450,160,79,1
4,BENE24402,CLM32911,PRV55912,3000,PHY380413,PHY432598,None,1748,1068.0,593,...,1,2,2,2,3000,1068,8900,740,84,1


In [ ]:
train_inPatientMergedWithProviderFraud_df.shape

(40474, 50)

In [ ]:
train_inPatientMergedWithProviderFraud_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40474 entries, 0 to 40473
Data columns (total 50 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   BeneID                           40474 non-null  object 
 1   ClaimID                          40474 non-null  object 
 2   Provider                         40474 non-null  object 
 3   InscClaimAmtReimbursed           40474 non-null  int64  
 4   AttendingPhysician               40474 non-null  object 
 5   OperatingPhysician               40474 non-null  object 
 6   OtherPhysician                   40474 non-null  object 
 7   ClmAdmitDiagnosisCode            40474 non-null  object 
 8   DeductibleAmtPaid                40474 non-null  float64
 9   DiagnosisGroupCode               40474 non-null  object 
 10  ClmDiagnosisCode_1               40474 non-null  object 
 11  ClmDiagnosisCode_2               40474 non-null  object 
 12  ClmDiagnosisCode_3

In [ ]:
train_inPatientMergedWithProviderFraud_df = train_inPatientMergedWithProviderFraud_df.apply(label_encoder.fit_transform)
train_inPatientMergedWithProviderFraud_df.head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,...,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age,PotentialFraud
0,2141,12665,1543,66,7483,0,0,1464,1,154,...,0,1,0,0,1863,30,12,7,40,1
1,13992,3045,1543,59,3649,4028,1253,754,1,730,...,0,1,1,1,1064,1,16,2,70,1
2,14845,33478,1543,55,2212,1609,0,1910,1,702,...,0,0,1,1,945,1,111,54,61,1
3,15126,34313,1543,49,7462,851,1253,1272,1,399,...,0,0,0,1,1486,16,51,16,53,1
4,15446,1549,1543,23,6519,8177,0,120,1,459,...,0,1,1,1,117,1,844,74,58,1


In [ ]:
train_outPatientMergedWithProviderFraud_df = pd.merge(train_outPatientMerged, train_providerFraud_df, on = 'Provider', how = 'inner')
train_outPatientMergedWithProviderFraud_df.head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,...,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age,PotentialFraud
0,BENE11002,CLM624349,PRV56011,30,PHY326117,None,None,78943,V5866,V1272,...,2,2,2,2,0,0,30,50,73,1
1,BENE11004,CLM121801,PRV56011,40,PHY334319,None,None,71988,0,0,...,1,1,1,2,0,0,1810,760,87,1
2,BENE11004,CLM150998,PRV56011,200,PHY403831,None,None,82382,30000,72887,...,1,1,1,2,0,0,1810,760,87,1
3,BENE11004,CLM173224,PRV56011,20,PHY339887,None,None,20381,0,0,...,1,1,1,2,0,0,1810,760,87,1
4,BENE11004,CLM224741,PRV56011,40,PHY345721,None,None,V6546,4280,2449,...,1,1,1,2,0,0,1810,760,87,1


In [ ]:
train_outPatientMergedWithProviderFraud_df.shape

(517737, 46)

In [ ]:
train_outPatientMergedWithProviderFraud_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 517737 entries, 0 to 517736
Data columns (total 46 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   BeneID                           517737 non-null  object 
 1   ClaimID                          517737 non-null  object 
 2   Provider                         517737 non-null  object 
 3   InscClaimAmtReimbursed           517737 non-null  int64  
 4   AttendingPhysician               517737 non-null  object 
 5   OperatingPhysician               517737 non-null  object 
 6   OtherPhysician                   517737 non-null  object 
 7   ClmDiagnosisCode_1               517737 non-null  object 
 8   ClmDiagnosisCode_2               517737 non-null  object 
 9   ClmDiagnosisCode_3               517737 non-null  object 
 10  ClmDiagnosisCode_4               517737 non-null  object 
 11  ClmDiagnosisCode_5               517737 non-null  object 
 12  Cl

In [ ]:
train_outPatientMergedWithProviderFraud_df = train_outPatientMergedWithProviderFraud_df.apply(label_encoder.fit_transform)
train_outPatientMergedWithProviderFraud_df.head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,...,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age,PotentialFraud
0,9080,413702,3726,3,9159,0,0,7243,4856,4033,...,1,1,1,1,10,0,3,5,47,1
1,9099,9430,3726,4,14065,0,0,6351,0,0,...,0,0,0,1,10,0,181,76,61,1
2,9099,32966,3726,20,55907,0,0,7913,962,2771,...,0,0,0,1,10,0,181,76,61,1
3,9099,50849,3726,2,17416,0,0,1278,0,0,...,0,0,0,1,10,0,181,76,61,1
4,9099,92238,3726,4,20943,0,0,10116,1831,479,...,0,0,0,1,10,0,181,76,61,1


In [ ]:
train_inPatientMergedWithProviderFraud_df = train_inPatientMergedWithProviderFraud_df.drop(columns = ['BeneID', 'ClaimID', 'Provider'])
train_outPatientMergedWithProviderFraud_df = train_outPatientMergedWithProviderFraud_df.drop(columns = ['BeneID', 'ClaimID', 'Provider'])
print(train_inPatientMergedWithProviderFraud_df.shape, train_outPatientMergedWithProviderFraud_df.shape)

(40474, 47) (517737, 43)


In [ ]:
labels = train_inPatientMergedWithProviderFraud_df['PotentialFraud']
features = train_inPatientMergedWithProviderFraud_df.drop('PotentialFraud', axis = 1)

feature_list = list(features.columns)

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [ ]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (30355, 46)
Training Labels Shape: (30355,)
Testing Features Shape: (10119, 46)
Testing Labels Shape: (10119,)


In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_features, train_labels)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [ ]:
lrPredictions = lr.predict(test_features)

In [ ]:
lrPredictions

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score

print('Recall:', recall_score(test_labels, lrPredictions) * 100)
print('Precision:', precision_score(test_labels, lrPredictions) * 100)
print('F1 Score:', f1_score(test_labels, lrPredictions) * 100)

Recall: 94.11764705882352
Precision: 58.10436634717785
F1 Score: 71.85092513333773


In [ ]:
  import xgboost as xgb

In [ ]:
model = xgb.XGBClassifier(max_depth=4,subsample=1,n_estimators=10000,learning_rate=0.1,min_child_weight=1,random_state=5,reg_alpha=0,reg_lambda=1)
model.fit(train_features, train_labels)
print(); print(model)


XGBClassifier(max_depth=4, n_estimators=10000, random_state=5)


In [ ]:
 expected_y  = test_labels
 predicted_y = model.predict(test_features)

In [ ]:
from sklearn import metrics

In [ ]:
print(metrics.classification_report(expected_y, predicted_y))
print(metrics.confusion_matrix(expected_y, predicted_y))

              precision    recall  f1-score   support

           0       0.78      0.75      0.77      4322
           1       0.82      0.84      0.83      5797

    accuracy                           0.81     10119
   macro avg       0.80      0.80      0.80     10119
weighted avg       0.80      0.81      0.80     10119

[[3251 1071]
 [ 900 4897]]
